### Calculate the density of tourist attractions in each LSOA in London 

In [17]:
!pip install --upgrade osmnx

In [18]:
import osmnx as ox
print(ox.__version__)  # Should be 1.2.2 or newer

2.0.0


In [19]:
import geopandas as gpd
import osmnx as ox
import pandas as pd

# Step 1: Load London LSOA boundary data
lsoa_boundaries = gpd.read_file("data/London/LSOA_2011_London_gen_MHW.shp")

# Step 2: Retrieve tourist attraction data for London
# Use OSMnx to extract nodes tagged as `tourism=attraction`
tourist_attractions = ox.features_from_place(
    "London, England", tags={"tourism": "attraction"}
)

# Ensure the coordinate reference system (CRS) is consistent
tourist_attractions = tourist_attractions.to_crs(lsoa_boundaries.crs)

# Step 3: Count the number of tourist attractions within each LSOA
lsoa_boundaries["tourist_attraction_count"] = lsoa_boundaries.apply(
    lambda row: tourist_attractions.within(row.geometry).sum(), axis=1
)

# Step 4: Calculate the density of tourist attractions (per square kilometer)
# Transform to an equal-area projection to compute accurate area
lsoa_boundaries = lsoa_boundaries.to_crs({"proj": "cea"})
lsoa_boundaries["area_km2"] = lsoa_boundaries.geometry.area / 1e6  # Convert to square kilometers

# Tourist attraction density = count / area
lsoa_boundaries["tourist_attraction_density"] = (
    lsoa_boundaries["tourist_attraction_count"] / lsoa_boundaries["area_km2"]
)

# Step 5: Export results to a CSV file
# Use 'LSOA11CD' as the LSOA code column
output_path = "data/london_lsoa_tourist_density.csv"
output_csv = lsoa_boundaries[["LSOA11CD", "tourist_attraction_count", "tourist_attraction_density"]]
output_csv.to_csv(output_path, index=False)

print(f"Tourist attraction density has been saved to {output_path}")

Tourist attraction density has been saved to data/london_lsoa_tourist_density.csv


### Transfer rental price data

In [5]:
import pandas as pd

# 转换为UTF-8编码并保存
with open("data/rental_price_postcode.csv", "r", encoding="ISO-8859-1") as f:
    content = f.read()

with open("data/rental_price_postcode.csv", "w", encoding="utf-8") as f:
    f.write(content)

# 加载房租数据
rental_data = pd.read_csv("data/rental_price_postcode.csv")

# 清洗数据：确保Postcode存在并无空值
rental_data = rental_data.dropna(subset=['Postcode'])
rental_data['Postcode'] = rental_data['Postcode'].astype(str).str.strip()

# 定义伦敦的Postcode前缀
london_postcodes = ['E', 'W', 'N', 'NW', 'SE', 'SW', 'EC', 'WC', 'BR', 'CR', 'DA', 'EN', 'HA', 'IG', 'KT', 'RM', 'SM', 'TW', 'UB', 'WD']

# 筛选伦敦的Postcode
rental_data['London'] = rental_data['Postcode'].str.startswith(tuple(london_postcodes))

# 提取伦敦数据
london_rentals = rental_data[rental_data['London']]

# 筛选所需的列
columns_to_keep = ['Postcode', 'Avg asking rent (pm)', 'Avg. household income']
london_rentals_filtered = london_rentals[columns_to_keep]

# 输出结果
print(london_rentals_filtered)

# 保存结果到新的CSV文件
london_rentals_filtered.to_csv("data/london_rental_filtered.csv", index=False, encoding="utf-8")
print("筛选后的伦敦租金数据已保存！")

     Postcode Avg asking rent (pm) Avg. household income
666     BR1 1                1,501                67,100
667     BR1 2                2,000                68,200
668     BR1 3                1,650                62,800
669     BR1 4                1,813                51,600
670     BR1 5                  NaN                46,900
...       ...                  ...                   ...
7345    WV6 9                  NaN                45,600
7346    WV7 3                  NaN                48,900
7347    WV8 1                  NaN                40,200
7348    WV8 2                  NaN                51,300
7349    WV9 5                  NaN                39,300

[2025 rows x 3 columns]
筛选后的伦敦租金数据已保存！


 ### Calculates hotel density for each LSOA 

In [21]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 1. Load LSOA boundary data
lsoa_shp_path = "data/London/LSOA_2011_London_gen_MHW.shp"  
lsoa_data = gpd.read_file(lsoa_shp_path)  # Load shapefile into GeoDataFrame
lsoa_data = lsoa_data.to_crs("EPSG:4326")  # Ensure the CRS is WGS84 for compatibility with OSM data

# Calculate the area of each LSOA in square kilometers
lsoa_data['area_km2'] = lsoa_data['geometry'].area / 10**6

# 2. Get the London boundary
london_boundary = lsoa_data.geometry.unary_union  # Merge all LSOA boundaries into a single polygon

# 3. Fetch hotel data from OpenStreetMap
# Use OSMnx's features_from_polygon method
hotels = ox.features_from_polygon(london_boundary, tags={"tourism": "hotel"})

# Convert the hotel data into a GeoDataFrame and retain only geometry
hotels_gdf = gpd.GeoDataFrame(hotels[['geometry', 'name']].dropna(subset=['geometry']), crs="EPSG:4326")

# 4. Spatially join hotels to their respective LSOA units
# Use GeoPandas's sjoin method
hotels_in_lsoa = gpd.sjoin(hotels_gdf, lsoa_data, how="inner", predicate="within")

# Count the number of hotels per LSOA
hotel_counts = hotels_in_lsoa.groupby('LSOA11CD').size().reset_index(name='hotel_count')

# 5. Calculate hotel density (number of hotels per square kilometer)
# Merge hotel counts back into the LSOA data
lsoa_data = lsoa_data.merge(hotel_counts, on='LSOA11CD', how='left')
lsoa_data['hotel_count'] = lsoa_data['hotel_count'].fillna(0)  # Fill missing values with 0
lsoa_data['hotel_density'] = lsoa_data['hotel_count'] / lsoa_data['area_km2']

# 6. Save results to a CSV file
output_csv_path = "data/lsoa_hotel_density.csv"  # Path for the output CSV file
lsoa_data[['LSOA11CD', 'hotel_count', 'area_km2', 'hotel_density']].to_csv(output_csv_path, index=False)
print(f"Hotel density data saved to {output_csv_path}")


/tmp/ipykernel_186/3352535683.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lsoa_data['area_km2'] = lsoa_data['geometry'].area / 10**6
/tmp/ipykernel_186/3352535683.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  london_boundary = lsoa_data.geometry.unary_union  # Merge all LSOA boundaries into a single polygon


Hotel density data saved to data/lsoa_hotel_density.csv


### Calculating five-year house price changes

In [22]:
import pandas as pd

file_path_xlsx = "data/LSOA_Median_properties_price.xlsx"

# Load the data
data_xlsx = pd.read_excel(file_path_xlsx, engine='openpyxl')

# List of all London boroughs including the City of London
london_boroughs = [
    "City of London", "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley", "Camden",
    "Croydon", "Ealing", "Enfield", "Greenwich", "Hackney", "Hammersmith and Fulham", "Haringey",
    "Harrow", "Havering", "Hillingdon", "Hounslow", "Islington", "Kensington and Chelsea",
    "Kingston upon Thames", "Lambeth", "Lewisham", "Merton", "Newham", "Redbridge",
    "Richmond upon Thames", "Southwark", "Sutton", "Tower Hamlets", "Waltham Forest", "Wandsworth", "Westminster"
]

# Filter the data for London based on the local authority name
london_data = data_xlsx[data_xlsx['Local authority name'].isin(london_boroughs)]

# Save the filtered data to a CSV file
output_csv_path = "data/London_Housing_Data.csv"
london_data.to_csv(output_csv_path, index=False)

print(f"Filtered London data has been saved to {output_csv_path}")

Filtered London data has been saved to data/London_Housing_Data.csv


In [27]:
import pandas as pd

# Step 1: Load the data
data = pd.read_csv('data/London_Housing_Data.csv')

# Step 2: Check the column names to ensure they are correct
print(data.columns)  # This will print all column names

# Strip any leading/trailing spaces in the column names
data.columns = data.columns.str.strip()

# Step 3: Convert 'Year ending Mar 2023' and 'Year ending Dec 2018' to numeric (if they are not already)
data['Year ending Mar 2023'] = pd.to_numeric(data['Year ending Mar 2023'], errors='coerce')
data['Year ending Dec 2018'] = pd.to_numeric(data['Year ending Dec 2018'], errors='coerce')

# Check for missing values in the year columns after conversion
missing_values = data[['Year ending Mar 2023', 'Year ending Dec 2018']].isnull().sum()
print(f"Missing values in Year columns:\n{missing_values}")

# Optionally, drop rows with missing values for price calculations (only for the year columns)
data_clean = data.dropna(subset=['Year ending Mar 2023', 'Year ending Dec 2018'])

# Step 4: Calculate the price change rate between 'Year ending Mar 2023' and 'Year ending Dec 2018'
data_clean['Price_Change'] = (data_clean['Year ending Mar 2023'] - data_clean['Year ending Dec 2018']) / data_clean['Year ending Dec 2018'] * 100

# Step 5: Merge the price change back with the original data (using LSOA_code as the key)
data_with_change = pd.merge(data, data_clean[['LSOA code', 'Price_Change']], on='LSOA code', how='left')

# Step 6: Ensure data is sorted by LSOA code
data_with_change_sorted = data_with_change.sort_values(by=['LSOA code'])

# Step 7: Save the result to a new CSV file
data_with_change_sorted[['LSOA code', 'Price_Change']].to_csv('data/LSOA_London_houseprice_change.csv', index=False)

print("House price change data saved to 'LSOA_London_houseprice_change.csv'")

Index(['Local authority code', 'Local authority name', 'LSOA code',
       'LSOA name', 'Year ending Dec 1995', 'Year ending Mar 1996',
       'Year ending Jun 1996', 'Year ending Sep 1996', 'Year ending Dec 1996',
       'Year ending Mar 1997',
       ...
       'Year ending Dec 2020', 'Year ending Mar 2021', 'Year ending Jun 2021',
       'Year ending Sep 2021', 'Year ending Dec 2021', 'Year ending Mar 2022',
       'Year ending Jun 2022', 'Year ending Sep 2022', 'Year ending Dec 2022',
       'Year ending Mar 2023'],
      dtype='object', length=114)
Missing values in Year columns:
Year ending Mar 2023    386
Year ending Dec 2018    359
dtype: int64
House price change data saved to 'LSOA_London_houseprice_change.csv'


/tmp/ipykernel_186/705826425.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Price_Change'] = (data_clean['Year ending Mar 2023'] - data_clean['Year ending Dec 2018']) / data_clean['Year ending Dec 2018'] * 100


In [25]:
# Define paths for the input Excel files and output CSV files
input_file_imd = "data/LSOA_IMD_london.xlsx"
output_file_imd = "data/LSOA_IMD_london.csv"

input_file_population = "data/LSOA_population_density.xlsx"
output_file_population = "data/LSOA_population_density.csv"

# Function to convert an Excel file to a CSV file
def convert_excel_to_csv(input_path, output_path):
    # Read the Excel file
    data = pd.read_excel(input_path)
    # Save the data to a CSV file
    data.to_csv(output_path, index=False)
    print(f"Converted {input_path} to {output_path}")

# Convert LSOA_IMD_london.xlsx to LSOA_IMD_london.csv
convert_excel_to_csv(input_file_imd, output_file_imd)

# Convert LSOA_population_density.xlsx to LSOA_population_density.csv
convert_excel_to_csv(input_file_population, output_file_population)

Converted data/LSOA_IMD_london.xlsx to data/LSOA_IMD_london.csv
Converted data/LSOA_population_density.xlsx to data/LSOA_population_density.csv


In [53]:
airbnb_df = pd.read_parquet("./20240614-London-listings.parquet")  # 直接使用下载的 Parquet 文件

SyntaxError: invalid syntax (2045445874.py, line 1)